# Working Example

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
from utils import VideoHandler, write_action_timestamp_csv
from optic_flow import OpticFlowCalculatorLK
from sync_videos import OffsetCalculator
from gaze_mapper import ActionCameraGazeMapper 
from video_renderer import save_video

Give a the path to the directory of the uncompressed Scene Video + Timeseries download from Cloud and its corresponding Action Camera recording ( please make sure that both videos have the same orientation):

In [ ]:
neon_timeseries_path='/users/sof/video_examples/second_video/2024-05-23_16-47-35-a666ea62'
neon_vid_path='/users/sof/video_examples/second_video/2024-05-23_16-47-35-a666ea62/e69049ea_0.0-42.986.mp4'
action_vid_path='/users/sof/video_examples/second_video/20240523_171941_000.mp4'

Define an output directory where the optic flow signal, gazes mapped to the action camera and the video visualization will be saved

In [ ]:
output_dir='/users/sof/action_map_experiments/second_video_2'

## Step 1: Obtain Optic Flow

There are two different optic_flow classes: one using Lucas-Kanade method (sparse-gridlike optic flow)and another one using  Gunnar Farneback method (dense, pixel-wise optic flow).

Lukas Kanade is way faster than the Gunnar Farneback Method, so this notebook will run with this method (For a 2min video it takes around ~3min)

In [ ]:
action_of = OpticFlowCalculatorLK(video_dir=action_vid_path)
neon_of = OpticFlowCalculatorLK(video_dir=neon_vid_path)

neon_result = neon_of.process_video()
action_result = action_of.process_video()

plt.figure(figsize=(25,5)) 
plt.plot(neon_result['end'].values,np.rad2deg(neon_result['angle'].values),label='neon')
plt.plot(action_result['end'].values,np.rad2deg(action_result['angle'].values),label='action')

With the write_to_csv method, the optic flow results are saved to a csv file in a user indicated address. The csv file will contain the following columns:
- start: the start frame timestamp of the optic flow calculation
- end: the end frame timestamp of the optic flow calculation
- angle
- avg_displacement_x
- avg_displacement_y

In [ ]:
optic_flow_output_dir = Path(output_dir, 'optic_flow')
optic_flow_output_dir.mkdir(parents=True, exist_ok=True)

action_of_csv=Path(optic_flow_output_dir, 'action_lk_of.csv')
neon_of_csv=Path(optic_flow_output_dir, 'neon_lk_of.csv')
action_of.write_to_csv(action_of_csv)
neon_of.write_to_csv(neon_of_csv)

## Step 2: Estimate time offset between videos

After optic flow is calculated, the OffsetCalculator class obtains the time offset of the action video with respect to the neon scene video. Optionally one can also indicate a period of time of the action (source) optic flow signal, to be matched to the whole neon (destination) signal 


In [ ]:
offset_calc=OffsetCalculator(src=action_result['dy'].values,src_timestamps=action_result['start'].values, dst=neon_result['dy'].values, dst_timestamps=neon_result['start'].values,resampling_frequency=500)
t_offset, pearson_corr = offset_calc.estimate_time_offset()
print(f'Estimated offset: {t_offset} seconds (Pearson correlation: {pearson_corr})')

start,end = None,None
idx=offset_calc._obtain_indexes(offset_calc.src_resampled_timestamps,start,end)
x_cor,lags=offset_calc._cross_correlate(offset_calc.src_resampled[idx[0]:idx[1]],offset_calc.dst_resampled)
s_offset = lags[np.argmax(x_cor)]
s,d=offset_calc._obtain_overlapping_signals(s_offset,offset_calc.src_resampled[idx[0]:idx[1]])
plt.figure(figsize=(25,5))
plt.plot(d,label='Neon')
plt.plot(s, label='Action')
plt.legend()


One can also select a time interval in the action video to estimate the delay. Try modifying the values in start_action and end_action. The printed estimated offset must be similar. If the visualization in the graph dpesnt seem to overlap nicely, try picking other values

In [ ]:
start_action = 2
end_action = 10

t_offset,pearson_corr = offset_calc.estimate_time_offset(source_end_time=end_action, source_start_time=start_action)
print(f'Estimated offset: {t_offset} seconds (Pearson correlation: {pearson_corr})')
idx=offset_calc._obtain_indexes(offset_calc.src_resampled_timestamps,start_action,end_action)
x_cor,lags=offset_calc._cross_correlate(offset_calc.src_resampled[idx[0]:idx[1]],offset_calc.dst_resampled)
s_offset = lags[np.argmax(x_cor)]
s,d=offset_calc._obtain_overlapping_signals(s_offset,offset_calc.src_resampled[idx[0]:idx[1]])
plt.figure(figsize=(25,5))
plt.plot(d,label='Neon')
plt.plot(s, label='Action')
plt.legend()

## Step 3: Creation of action_worldtimestamps.csv

Once calculated the time offset, the action_worldtimestamps.csv can be generated with the function write_worldtimestamp_csv. This function wirtes the csv to the same directory as the given world_timestamps.csv from Neon

In [ ]:
actionVid=VideoHandler(action_vid_path)
neon_timestamps= Path(neon_timeseries_path, 'world_timestamps.csv')
action_relative_ts= actionVid.timestamps
write_action_timestamp_csv(neon_timestamps, action_relative_ts+t_offset)

## Step 4: Map gaze data to the action video

In [ ]:
action_timestamps= Path(neon_timeseries_path, 'action_camera_timestamps.csv')

neon_gaze_csv=Path(neon_timeseries_path, 'gaze.csv')

param_lg = {'num_features':2048,'gpu_num':0}
param_loftr = {'location':'outdoor', 'gpu_num':0}
image_matcher_loftr={'choice':'loftr','parameters':param_loftr}
image_matcher_lg={'choice':'disk_lightglue','parameters':param_lg}

image_matcher=image_matcher_lg

In [ ]:
mapper = ActionCameraGazeMapper(neon_gaze_csv=neon_gaze_csv,
        neon_video_dir=neon_vid_path,
        action_video_dir=action_vid_path,
        neon_worldtimestamps=neon_timestamps,
        action_worldtimestamps=action_timestamps,
        image_matcher=image_matcher['choice'],
        image_matcher_parameters=image_matcher['parameters'],
        neon_opticflow_csv=neon_of_csv,
        action_opticflow_csv=action_of_csv,
        patch_size=1000)

In [ ]:
action_gaze_output_dir = Path(output_dir, f'mapped_gaze/{image_matcher["choice"].lower()}')
action_gaze_output_dir.mkdir(parents=True, exist_ok=True)
gaze_csv_path = Path(action_gaze_output_dir,f"action_gaze_lnk.csv")
mapper.map_gaze(saving_path=gaze_csv_path)

## Step 5: Create a video with both Neon and Action Camera 

In [ ]:
action_gaze_dict = {image_matcher['choice'].upper(): gaze_csv_path}
rendered_video_path = Path(output_dir, f"video_rendered/Neon_Actionrendered_{image_matcher['choice']}_lk.avi")
Path(rendered_video_path).parent.mkdir(parents=True, exist_ok=True)

In [ ]:
save_video(action_video_path=action_vid_path,
        action_worldtimestamps_path=action_timestamps,
        action_gaze_paths_dict=action_gaze_dict,
        neon_video_path=neon_vid_path,
        neon_worldtimestamps_path=neon_timestamps,
        neon_gaze_path=neon_gaze_csv,
        save_video_path=rendered_video_path)  